# Création d'une ontologie

Pour ce faire, nous utiliserons le package `owlready2` qui permet de créer des ontologies en python. (https://linuxfr.org/news/owlready-un-module-python-pour-manipuler-les-ontologies-owl)
Les données seront extraite de la base de données de WikiData, nous interrogerons cette base de données par l'API mis à disposition par la librairie `wikibaseintegrator` (https://github.com/LeMyst/WikibaseIntegrator#execute-sparql-queries).

Commençons par installer les packages nécessaires

In [ ]:
# !pip install owlready2
# !pip install wikibaseintegrator

## Récupération des données depuis une base de données : WikiData

Pour ce faire, nous utiliserons la librairie `wikibaseintegrator` qui permet d'interroger la base de données de WikiData.
Nous voulons récupérer l'ensemble des animaux de la base de données, pour ce faire, nous utiliserons : WikibaseIntegrator wbi.item.get(entity_id='Q729').

In [ ]:
from wikibaseintegrator import wbi_login, WikibaseIntegrator
from wikibaseintegrator.wbi_config import config as wbi_config

wbi_config['USER_AGENT'] = 'MyWikibaseBot/1.0 (https://www.wikidata.org/wiki/User:MyUsername)'

# login object
login_instance = wbi_login.Clientlogin(user='Arti0890', password='purtyx-fuwvi4-rYmgor')

wbi = WikibaseIntegrator(login=login_instance)

item = wbi.item.get(entity_id='Q729')

# to check successful installation and retrieval of the data, you can print the json representation of the item
print(item.get_json())

In [ ]:
from wikibaseintegrator import wbi_helpers

# Requête SPARQL pour récupérer les animaux
query = """
SELECT DISTINCT ?animal ?animalLabel WHERE {
  ?animal wdt:P31 wd:Q729.
  ?animal rdfs:label ?animalLabel filter (lang(?animalLabel) = "en").
  #OPTIONAL {?animal wdt:P18 ?image.}
} LIMIT 100
"""

# Récupérer les entités correspondant à la requête SPARQL
data = wbi_helpers.execute_sparql_query(query)
print(data)


## Création de l'ontologie

In [ ]:
import owlready2 as owl

onto = owl.get_ontology("http://example.com/animals.owl")

class Animalia(owl.Thing):
    namespace = onto

class Mammifere(Animalia):
    namespace = onto

class aPourNom(owl.DataProperty):
    namespace = onto
    domain = [Animalia]
    range = [str]

chien = Animalia("chien")
chien.aPourNom.append("Fido")

baleine = Mammifere("baleine")
baleine.aPourNom.append("Moby Dick")

onto.save(file="animaux.owl", format="rdfxml")

In [78]:
from owlready2 import Thing, AllDisjoint, ObjectProperty

import owlready2 as owl

onto = owl.get_ontology("http://example3.com/animalia.owl")

with onto:
    class Animalia(Thing):
        pass
    class Mammalia(Animalia): #Mamifères
        pass
    class Aves(Animalia): #Oiseaux
        pass
    class Pisces(Animalia): #Poissons
        pass

    # Définition de la propriété object property aUnBec
    class a_un_bec(ObjectProperty):
        domain = [Aves]
        range = [str]

    # Ajout d'une restriction de propriété pour définir les caractéristiques des oiseaux
    class Oiseau(Aves):
        equivalent_to = [Aves & a_un_bec.some(str)]

    #Un mamifère n'est pas un poisson, etc.
    AllDisjoint([Mammalia, Aves, Pisces])

my_oiseau = Oiseau("Moineau")
my_oiseau.a_un_bec = ["court et pointu"]

print(Oiseau.is_a)
print(my_oiseau.a_un_bec)

onto.save(file="animalia.owl", format="rdfxml")


[animalia.Aves, animalia.Animalia]
['court et pointu']
